In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\50510\Downloads\Dataset_Public.xlsx")
print(df.columns, df.shape)
df.head()

Index(['Claim Description', 'Coverage Code', 'Accident Source'], dtype='object') (191690, 3)


,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
1,CLAIMANT ALLEGES SHE SUFFERED INJURIES IN AN E...,GB,Elevator/Escalator
2,"IV PASSENGER SUSTAINED INJURIES, OV AND IV COL...",AB,Sideswipe or lane change
3,CLAIMANT ALLEGES SHE WAS BURNED UNKNOWN DEGREE...,PA,Food Product
4,THE IV WAS MERGING INTO A CONSTRUCTION ZONE WH...,AD,Struck vehicle in rear


In [2]:
# Clean the Data
df['Claim Description'] = df['Claim Description'].astype('str')

def clean(text):
    text = text.lower()
    return text

df['Claim Description'] = df['Claim Description'].apply(lambda x: clean(x))

In [3]:
print(df['Coverage Code'].nunique(), df['Accident Source'].nunique())

43 312


In [4]:
Coverage_Code = df['Coverage Code'].value_counts()
Coverage_Code = pd.DataFrame(Coverage_Code)
Coverage_Code.reset_index(inplace=True)
Coverage_Code

from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Fit the label encoder on the coverage labels and transform them to numerical values
numerical_labels = label_encoder.fit_transform(Coverage_Code['Coverage Code'])

# Add the numerical labels as a new column in the DataFrame
Coverage_Code['Coverage_Labels'] = numerical_labels
Coverage_Code.drop(columns=['count'], inplace=True)
Coverage_Code.head()

,Coverage Code,Coverage_Labels
0,AD,1
1,GB,19
2,GD,20
3,AP,5
4,AB,0


In [5]:
correction_dict = Coverage_Code.set_index("Coverage Code")["Coverage_Labels"].to_dict()

# Update the "CoverageCode_Labelss" column in the original data using the correction dictionary

df['CoverageCode_Labelss'] = df["Coverage Code"].map(correction_dict)

In [6]:
Accident = df['Accident Source'].value_counts()
Accident = pd.DataFrame(Accident)
Accident.reset_index(inplace=True)
Accident
# Fit the label encoder on the coverage labels and transform them to numerical values
numerical_labels1 = label_encoder.fit_transform(Accident['Accident Source'])

# Add the numerical labels as a new column in the DataFrame
Accident['Accident_Labels'] = numerical_labels1



correction_dict1 = Accident.set_index("Accident Source")["Accident_Labels"].to_dict()
correction_dict1
# Update the "Candidate_Name" column in the original data using the correction dictionary

df['Accident_Source_Label'] = df["Accident Source"].map(correction_dict1)

In [7]:
df.head()

,Claim Description,Coverage Code,Accident Source,CoverageCode_Labelss,Accident_Source_Label
0,the iv was making a left turn on a green arrow...,AN,"Struck pedestrian, bicycle",4,247
1,claimant alleges she suffered injuries in an e...,GB,Elevator/Escalator,19,90
2,"iv passenger sustained injuries, ov and iv col...",AB,Sideswipe or lane change,0,225
3,claimant alleges she was burned unknown degree...,PA,Food Product,31,104
4,the iv was merging into a construction zone wh...,AD,Struck vehicle in rear,1,248


In [8]:
df.isnull().sum()

Claim Description        0
Coverage Code            0
Accident Source          0
CoverageCode_Labelss     0
Accident_Source_Label    0
dtype: int64

In [9]:
df = df[0:10000]

In [10]:
X = df['Claim Description']
y_coverage_encoded = df['CoverageCode_Labelss']
y_source_encoded = df['Accident_Source_Label']
y_coverage_encoded.shape

(10000,)

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

#### Coverage

In [18]:
from imblearn.over_sampling import SMOTE, KMeansSMOTE , ADASYN,SVMSMOTE,KMeansSMOTE,BorderlineSMOTE, ADASYN, RandomOverSampler

X_train, X_test, y_train, y_test = train_test_split(X, y_coverage_encoded.values, test_size=0.30)

In [19]:
X_train.shape, X_test.shape

((7000,), (3000,))

TermFrequency = term(word)/total_words

InverseDocumentFrequency = total_text/term

In [20]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(X_train)

X_train_tf = vectorizer.transform(X_train)
X_train_tf = X_train_tf.toarray()
X_test_tf = vectorizer.transform(X_test)
X_test_tf = X_test_tf.toarray()

In [21]:
# over-sampling using SMOTE.
oversample=RandomOverSampler()

X_train_ros, y_train_ros = oversample.fit_resample(X_train_tf, y_train)


In [33]:
y_t = pd.DataFrame(y_train_ros)
y_t.value_counts()

0     2784
31    2784
23    2784
24    2784
25    2784
26    2784
28    2784
30    2784
32    2784
1     2784
34    2784
35    2784
36    2784
37    2784
38    2784
39    2784
22    2784
21    2784
20    2784
19    2784
17    2784
16    2784
15    2784
14    2784
11    2784
10    2784
8     2784
7     2784
6     2784
5     2784
4     2784
3     2784
2     2784
42    2784
Name: count, dtype: int64

In [22]:
## Random Forest

rd = RandomForestClassifier()
rd.fit(X_train_ros, y_train_ros)
y_preds = rd.predict(X_test_tf)
print(y_preds)
print(accuracy_score(y_test, y_preds))

[20 15  1 ... 19 19 31]
0.621


In [23]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.38      0.23      0.29       226
           1       0.59      0.76      0.67       797
           2       0.88      0.91      0.90       152
           3       0.14      0.20      0.17         5
           4       0.33      0.08      0.12        13
           5       0.49      0.35      0.41       355
           6       0.45      0.38      0.41        24
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         5
          14       1.00      0.17      0.29        12
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         5
          17       0.00      0.00      0.00         1
          19       0.68      0.88      0.77       561
          20       0.62      0.62      0.62       417
          21       0.00    

f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Source

In [12]:
from imblearn.over_sampling import SMOTE, KMeansSMOTE , ADASYN,SVMSMOTE,KMeansSMOTE,BorderlineSMOTE, ADASYN, RandomOverSampler

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y_source_encoded.values, test_size=0.30)

In [13]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(X_train1)

X_train_tf1 = vectorizer.transform(X_train1)
X_train_tf1 = X_train_tf1.toarray()
X_test_tf1 = vectorizer.transform(X_test1)
X_test_tf1 = X_test_tf1.toarray()


In [14]:

# over-sampling using SMOTE.
oversample=RandomOverSampler()

X_train_ros1, y_train_ros1 = oversample.fit_resample(X_train_tf1, y_train1)

In [17]:
## Random Forest

rd1 = RandomForestClassifier()
rd1.fit(X_train_ros1, y_train_ros1)
y_preds1 = rd1.predict(X_test_tf1)
print(y_preds1)
print(accuracy_score(y_test1, y_preds1))
print(classification_report(y_test1, y_preds1))

[  7   7 225 ... 185 152 225]
0.3913333333333333
              precision    recall  f1-score   support

           1       1.00      0.17      0.29         6
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         0
           6       0.50      0.50      0.50         4
           7       0.29      0.58      0.39       350
          10       0.00      0.00      0.00         1
          11       0.45      0.38      0.41        34
          12       0.10      0.06      0.08        32
          13       0.33      0.13      0.19        30
          14       0.37      0.50      0.42        34
          16       0.00      0.00      0.00         0
          17       0.00      0.00      0.00         4
          18       0.32      0.27      0.29        93
          19       0.00      0.00      0.00         2
          20       0.00      0.00      0.00         1
          22       0.00      0.0

f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
f:\Python 3.11\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarn

In [25]:
df.columns

Index(['Claim Description', 'Coverage Code', 'Accident Source',
       'CoverageCode_Labelss', 'Accident_Source_Label'],
      dtype='object')

In [26]:
import joblib

# Save the trained model to a file
model_filename = 'Accident_Source_model.pkl'
joblib.dump(rd1, model_filename)

['Accident_Source_model.pkl']

## Accident Source  Testing

In [27]:
loaded_model = joblib.load(r"C:\Users\50510\Desktop\MAR IHTOYJ\Accident_Source_model.pkl")
loaded_model

RandomForestClassifier()

In [29]:
text = ["CRACKED WINDSHIELD"]

tetss = vectorizer.transform(text)
tetss = tetss.toarray()
tetss

array([[0., 0., 0., ..., 0., 0., 0.]])

In [33]:
predicted_labels = loaded_model.predict(tetss)
predicted_labels

array([12], dtype=int64)

In [31]:
Accident.head()

,Accident Source,count,Accident_Labels
0,Alleged Negligent Act,21284,7
1,Sideswipe or lane change,19220,225
2,Struck vehicle in rear,10676,248
3,Not Otherwise Classified,8359,185
4,Struck animal or object,8195,238


In [32]:
numerical_to_accident_mapping = Accident.set_index("Accident_Labels")["Accident Source"].to_dict()
numerical_to_accident_mapping

{7: 'Alleged Negligent Act',
 225: 'Sideswipe or lane change',
 248: 'Struck vehicle in rear',
 185: 'Not Otherwise Classified',
 238: 'Struck animal or object',
 246: 'Struck parked vehicle',
 44: 'Backed into vehicle or object',
 152: 'Intersection accident',
 197: 'Our vehicle struck in rear',
 18: 'Alleged improper maintenance - other',
 211: 'Pothole',
 311: 'e-Commerce',
 14: 'Alleged foreign object in product',
 292: 'Vehicle Accident',
 59: 'Cart',
 305: 'Windshield',
 118: 'Ground/floor',
 12: 'Alleged damage to property of others',
 136: 'Human Action, NOC',
 249: 'Struck/pulled down wires',
 11: 'Alleged contamination or spoilage',
 13: 'Alleged design flaw, defect',
 191: 'Other - comprehensive',
 104: 'Food Product',
 239: 'Struck building, overhang',
 77: 'Damage to landscaping',
 257: 'Theft',
 217: 'Road debris/road hazard',
 198: 'Our vehicle struck while parked',
 205: 'Parking lot, driveway',
 141: 'Ice/water',
 31: 'Alleged product loss, NOC',
 33: 'Alleged service 

In [35]:
# Map predicted numerical labels to Accident Source using the mapping
predicted_sources = [numerical_to_accident_mapping[label] for label in predicted_labels]
predicted_sources

['Alleged damage to property of others']

## Coverage_Code  Testing

In [36]:
import joblib

# Save the trained model to a file
model_filename = 'Coverage_Code_model.pkl'
joblib.dump(rd, model_filename)

['Coverage_Code_model.pkl']

In [37]:
loaded_Coverage_Code = joblib.load(r"C:\Users\50510\Desktop\MAR IHTOYJ\Coverage_Code_model.pkl")
loaded_Coverage_Code

RandomForestClassifier()

In [38]:
predicted_labels = loaded_Coverage_Code.predict(tetss)
predicted_labels

array([5], dtype=int64)

In [39]:
Coverage_Code.head()

,Coverage Code,Coverage_Labels
0,AD,1
1,GB,19
2,GD,20
3,AP,5
4,AB,0


In [41]:
numerical_to_code_mapping = Coverage_Code.set_index("Coverage_Labels")["Coverage Code"].to_dict()
numerical_to_code_mapping

{1: 'AD',
 19: 'GB',
 20: 'GD',
 5: 'AP',
 0: 'AB',
 2: 'AL',
 31: 'PA',
 32: 'PB',
 38: 'RB',
 28: 'NS',
 6: 'AU',
 36: 'PM',
 14: 'EL',
 4: 'AN',
 35: 'PL',
 21: 'GK',
 39: 'RC',
 34: 'PI',
 23: 'GO',
 3: 'AM',
 26: 'LL',
 11: 'DC',
 22: 'GL',
 16: 'EP',
 24: 'IK',
 30: 'OM',
 15: 'EO',
 37: 'PP',
 42: 'TE',
 25: 'IM',
 8: 'BM',
 7: 'BL',
 17: 'FB',
 10: 'CM',
 40: 'RF',
 29: 'OI',
 9: 'BR',
 41: 'RQ',
 12: 'EB',
 18: 'FF',
 27: 'LS',
 13: 'EI',
 33: 'PC'}

In [42]:
# Map predicted numerical labels to Accident Source using the mapping
predicted_code = [numerical_to_code_mapping[label] for label in predicted_labels]
predicted_code

['AP']

In [43]:
import pickle

# Save the dictionary to a file
with open('numerical_to_code_mapping.pkl', 'wb') as f:
    pickle.dump(numerical_to_code_mapping, f)

# Load the dictionary from the file
with open('numerical_to_code_mapping.pkl', 'rb') as f:
    loaded_numerical_to_code_mapping = pickle.load(f)


In [47]:
# Load the dictionary from the file
with open('numerical_to_accident_mapping.pkl', 'rb') as f:
    numerical_to_accident_mapping = pickle.load(f)
numerical_to_accident_mapping

{7: 'Alleged Negligent Act',
 225: 'Sideswipe or lane change',
 248: 'Struck vehicle in rear',
 185: 'Not Otherwise Classified',
 238: 'Struck animal or object',
 246: 'Struck parked vehicle',
 44: 'Backed into vehicle or object',
 152: 'Intersection accident',
 197: 'Our vehicle struck in rear',
 18: 'Alleged improper maintenance - other',
 211: 'Pothole',
 311: 'e-Commerce',
 14: 'Alleged foreign object in product',
 292: 'Vehicle Accident',
 59: 'Cart',
 305: 'Windshield',
 118: 'Ground/floor',
 12: 'Alleged damage to property of others',
 136: 'Human Action, NOC',
 249: 'Struck/pulled down wires',
 11: 'Alleged contamination or spoilage',
 13: 'Alleged design flaw, defect',
 191: 'Other - comprehensive',
 104: 'Food Product',
 239: 'Struck building, overhang',
 77: 'Damage to landscaping',
 257: 'Theft',
 217: 'Road debris/road hazard',
 198: 'Our vehicle struck while parked',
 205: 'Parking lot, driveway',
 141: 'Ice/water',
 31: 'Alleged product loss, NOC',
 33: 'Alleged service 

In [45]:
# Save the dictionary to a file
with open('numerical_to_accident_mapping.pkl', 'wb') as f:
    pickle.dump(numerical_to_accident_mapping, f)